In [2]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt

## upload out data set, separate initial X,Y locations and correspnonding power outputs

file_npath = "/content/WEC_Perth_49.csv"

df = pd.read_csv(file_npath)
df_sample = df.sample(n=1000, random_state=42).reset_index(drop=True)

data = torch.tensor(df_sample.values, dtype=torch.float32)
print(data.size())

XY_s = data[:, :98]
Power_s = data[:, 98:147]
qW_s = data[:, 147]
total_Power_s = data[:, 148]

## sum of all Power_s for an instance = total_Power_s

## qW_s = total_Power_s / maximum_Power_s


# Train-test split for individual buoy power outputs
X_train, X_test, y_train_power, y_test_power = train_test_split(XY_s, Power_s, test_size=0.3, random_state=42)

# Train-test split for total power output
_, _, y_train_total, y_test_total = train_test_split(XY_s, total_Power_s, test_size=0.3, random_state=42)

print(f"Train features shape: {X_train.size()}, Train buoy power targets shape: {y_train_power.size()}, Train total power targets shape: {y_train_total.size()}")



torch.Size([1000, 149])
Train features shape: torch.Size([700, 98]), Train buoy power targets shape: torch.Size([700, 49]), Train total power targets shape: torch.Size([700])


In [10]:
import torch
import torch.nn as nn

class CustomLoss(nn.Module): ## establish our loss function
    def __init__(self, alpha=1.0):
        super(CustomLoss, self).__init__()
        self.alpha = alpha
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_train_total):
        """
        y_pred: Predicted buoy power outputs (batch_size x 49)
        y_train_power: True buoy power outputs (batch_size x 49)
        """
        # Compute MSE for individual buoy power outputs
        power_loss = self.mse(y_pred, y_train_total)

        # Combine losses with weights
        loss = self.alpha * power_loss
        return loss

In [4]:
from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=10)
tree_model.fit(X_train.numpy(), y_train_total.numpy())
y_pred_tree = tree_model.predict(X_test.numpy())


from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
RF_model.fit(X_train.numpy(), y_train_total.numpy())
y_pred_RF = RF_model.predict(X_test.numpy())


from xgboost import XGBRegressor

boost_model = XGBRegressor(n_estimators=100, max_depth=10, learning_rate=0.1)
boost_model.fit(X_train.numpy(), y_train_total.numpy())
y_pred_boost = boost_model.predict(X_test.numpy())


from sklearn.svm import SVR

SVR_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
SVR_model.fit(X_train.numpy(), y_train_total.numpy())
y_pred_SVR = SVR_model.predict(X_test.numpy())

from sklearn.neighbors import KNeighborsRegressor

KNN_model = KNeighborsRegressor(n_neighbors=5)
KNN_model.fit(X_train.numpy(), y_train_total.numpy())
y_pred_KNN = KNN_model.predict(X_test.numpy())


In [40]:
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Define the model, loss, and optimizer
input_size = X_train.size(1)
hidden_size = 64
output_size = 1  # For total power prediction
NN_model = NeuralNetwork(input_size, hidden_size, output_size)
loss_fn = CustomLoss(alpha=1.0)  # Adjust alpha and beta as needed
optimizer = optim.Adam(NN_model.parameters(), lr=0.001)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    NN_model.train()
    y_pred = NN_model(X_train)
    y_pred = y_pred.squeeze()  # Remove the extra dimension
    loss = loss_fn(y_pred, y_train_total)  # Add batch dimension
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# # Evaluation on test set
NN_model.eval()
with torch.no_grad():
    y_pred_test = NN_model(X_test).squeeze()
    test_loss = loss_fn(y_pred_test, y_test_total)
    print(f"Test Loss: {test_loss.item():.4f}")

Test Loss: 15175410253824.0000


In [38]:
loss_fn = CustomLoss(alpha=1.0)

print("Decision Tree: ")
y_pred_tree_tensor = torch.tensor(y_pred_tree, dtype=torch.float32)
test_loss = loss_fn(y_pred_tree_tensor, y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")

print("Random Forest: ")
y_pred_RF_tensor = torch.tensor(y_pred_RF, dtype=torch.float32)
test_loss = loss_fn(y_pred_RF_tensor, y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")

print("Boost: ")
y_pred_boost_tensor = torch.tensor(y_pred_boost, dtype=torch.float32)
test_loss = loss_fn(y_pred_boost_tensor, y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")

print("SVR: ")
y_pred_SVR_tensor = torch.tensor(y_pred_SVR, dtype=torch.float32)
test_loss = loss_fn(y_pred_SVR_tensor, y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")

print("KNN: ")
y_pred_KNN_tensor = torch.tensor(y_pred_KNN, dtype=torch.float32)
test_loss = loss_fn(y_pred_KNN_tensor, y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")

Decision Tree: 
Test Loss: 4782450176.0000
Random Forest: 
Test Loss: 2441870592.0000
Boost: 
Test Loss: 2361456896.0000
SVR: 
Test Loss: 14457721856.0000
KNN: 
Test Loss: 4775132672.0000


In [42]:
# Implements multi-output regression models for the task of individual buoy power output prediction.

from sklearn.tree import DecisionTreeRegressor

tree_model = DecisionTreeRegressor(max_depth=10)
tree_model.fit(X_train.numpy(), y_train_power.numpy())
y_pred_tree_multi = tree_model.predict(X_test.numpy())


from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
RF_model.fit(X_train.numpy(), y_train_power.numpy())
y_pred_RF_multi = RF_model.predict(X_test.numpy())


from xgboost import XGBRegressor

boost_model = XGBRegressor(n_estimators=100, max_depth=10, learning_rate=0.1)
boost_model.fit(X_train.numpy(), y_train_power.numpy())
y_pred_boost_multi = boost_model.predict(X_test.numpy())


from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR

# Wrap SVR in MultiOutputRegressor
SVR_model = MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
SVR_model.fit(X_train.numpy(), y_train_power.numpy())  # y_train_power shape: (batch_size, 49)
y_pred_SVR = SVR_model.predict(X_test.numpy())

from sklearn.neighbors import KNeighborsRegressor

KNN_model = KNeighborsRegressor(n_neighbors=5)
KNN_model.fit(X_train.numpy(), y_train_power.numpy())
y_pred_KNN = KNN_model.predict(X_test.numpy())

In [43]:
print("Decision Tree: ")
test_loss = nn.MSELoss()(torch.tensor(y_pred_tree_multi, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

print("Random Forest: ")
test_loss = nn.MSELoss()(torch.tensor(y_pred_RF_multi, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

print("Boost: ")
test_loss = nn.MSELoss()(torch.tensor(y_pred_boost_multi, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

print("SVR: ")
test_loss = nn.MSELoss()(torch.tensor(y_pred_SVR, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

print("KNN: ")
test_loss = nn.MSELoss()(torch.tensor(y_pred_KNN, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

Decision Tree: 
Test Loss: 64288484.0000
Random Forest: 
Test Loss: 40985464.0000
Boost: 
Test Loss: 24024452.0000
SVR: 
Test Loss: 126940368.0000
KNN: 
Test Loss: 47109600.0000


In [8]:
def compute_inter_buoy_distances(XY_s, num_buoys=49):
    distances = []
    for i in range(num_buoys):
        for j in range(i + 1, num_buoys):
            dist = torch.sqrt((XY_s[:, 2*i] - XY_s[:, 2*j])**2 + (XY_s[:, 2*i+1] - XY_s[:, 2*j+1])**2)
            distances.append(dist)
    return torch.stack(distances, dim=1)

# Compute inter-buoy distances and add to features
inter_buoy_distances_train = compute_inter_buoy_distances(X_train)
inter_buoy_distances_test = compute_inter_buoy_distances(X_test)

# Concatenate distances to XY features
X_train = torch.cat((X_train, inter_buoy_distances_train), dim=1)
X_test = torch.cat((X_test, inter_buoy_distances_test), dim=1)

print(f"Enhanced train features shape: {X_train.size()}, Test features shape: {X_test.size()}")


Enhanced train features shape: torch.Size([700, 1274]), Test features shape: torch.Size([300, 1274])


In [48]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

## Use randomized search to find best hyperparameters for predicting TOTAL power output

# Define model
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 1, 10],
    'reg_lambda': [1, 5, 10],
}

# Randomized Search
search = RandomizedSearchCV(xgb, param_distributions=param_grid, n_iter=100, cv=3, scoring='neg_mean_squared_error', verbose=1, random_state=42, n_jobs=-1)
search.fit(X_train.numpy(), y_train_total.numpy())

# Best model
best_xgb = search.best_estimator_
print("Best model:", best_xgb)

# Best parameters
print("Best parameters:", search.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best model: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
Best parameters: {'subsample': 0.8, 'reg_lambda': 10, 'reg_alpha': 1, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.1, 'gamma': 5, 'colsample_bytree': 1.0}


In [49]:

y_pred_best_boost = best_xgb.predict(X_test.numpy())
print("Best Boost: ")
test_loss = loss_fn(torch.tensor(y_pred_best_boost, dtype=torch.float32), y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")
#

Best Boost: 
Test Loss: 1594093056.0000


In [57]:
## Use randomized search to find best hyperparameters for predicting INDIVIDUAL power output

best_xgb = XGBRegressor(subsample=0.8, reg_lambda=10, reg_alpha=1, n_estimators=300, min_child_weight=1, max_depth=3, learning_rate=0.1, colsample_bytree=1.0, gamma=5, random_state=42)
best_xgb.fit(X_train.numpy(), y_train_power.numpy())
y_pred_best_boost_multi = best_xgb.predict(X_test.numpy())

print("Best Boost: ")
test_loss = loss_fn(torch.tensor(y_pred_best_boost_multi, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

Best Boost: 
Test Loss: 22498574.0000


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define model
rf = RandomForestRegressor(random_state=42)

# Hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
}

# Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train.numpy(), y_train_total.numpy())

# Best model
best_rf = grid_search.best_estimator_
print("Best model:", best_rf)

# Best parameters
print("Best parameters:", grid_search.best_params_)


Fitting 3 folds for each of 432 candidates, totalling 1296 fits


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best model: RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      n_estimators=300, random_state=42)
Best parameters: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [11]:
loss_fn = CustomLoss(alpha=1.0)

best_rf = RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      n_estimators=300, random_state=42)
best_rf.fit(X_train.numpy(), y_train_total.numpy())
y_pred_best_RF = best_rf.predict(X_test.numpy())

print("Best RF_total: ")
test_loss = loss_fn(torch.tensor(y_pred_best_RF, dtype=torch.float32), y_test_total)
print(f"Test Loss: {test_loss.item():.4f}")



## Now for individual power outputs

best_rf = RandomForestRegressor(bootstrap=False, max_depth=30, max_features='sqrt',
                      n_estimators=300, random_state=42)
best_rf.fit(X_train.numpy(), y_train_power.numpy())
y_pred_best_RF_multi = best_rf.predict(X_test.numpy())

print("Best RF_power: ")
test_loss = loss_fn(torch.tensor(y_pred_best_RF_multi, dtype=torch.float32), y_test_power)
print(f"Test Loss: {test_loss.item():.4f}")

Best RF_total: 
Test Loss: 2033772800.0000
Best RF_power: 
Test Loss: 35088712.0000
